In [1]:
#Importing Required Libraries 
import pandas as pd 
import numpy as np 
from numpy import nan as NA 
from pandas import DataFrame, Series 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score


In [2]:
#Reading Bank Dataset excel file
df=pd.read_csv('~/Downloads/MRP-22072018/bank/bank-full copy.csv')

In [3]:
#Converting Categorical columns to numerical
for columnName in df.columns:
    if(df[columnName].dtype == 'object'):
        df[columnName]= df[columnName].astype('category')
        df[columnName] = df[columnName].cat.codes

In [4]:
#Models built with 6 Variables 
#split of Train-Test
df2=df.copy(deep=True)
df2=df2[['housing','contact','duration','pdays','previous','poutcome', 'y', 'month']]
df_Backup=df2.copy(deep=True)

In [5]:
from sklearn.cross_validation import train_test_split
#label = df2.pop('y')
#data_train, data_test, label_train, label_test = train_test_split(df2, label, test_size = 0.2, random_state = 123)
data_train, data_test = train_test_split(df2, test_size = 0.2, random_state = 0)

/Users/harneetkaur/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
data_test_Backup=data_test.copy(deep=True)

In [7]:
# Upsampling for minor classes
from sklearn.utils import resample
df=data_train.copy(deep=True)

# Separate majority and minority classes
df_majority = df[df.y==0]
df_minority = df[df.y==1]
 
# Downsample majority class
df_majority_upsampled = resample(df_majority, replace=False, n_samples=31942, random_state=123) # reproducible results
 
# Combining minor class with downsampled major class
df_upsampled = pd.concat([df_majority_upsampled, df_minority])

data_train=df_upsampled.copy(deep=True)
df_Backup_train=data_train.copy(deep=True)

In [8]:
from sklearn.ensemble import RandomForestClassifier

df3=df_Backup_train.copy(deep=True)
label_df3=df3['y']
train_month=df3['month']
df3=df3.drop('month', axis=1)
df3=df3.drop('y', axis=1)

#print(df3.head())

rfc = RandomForestClassifier()
rfc.fit(df3, label_df3)
    
predictions = rfc.predict(df3)
    
df3['pred']=predictions
df3['month']=train_month

df3['y']=label_df3
print(df3.head())

       housing  contact  duration  pdays  previous  poutcome  pred  month  y
11139        1        2        34     -1         0         3     0      6  0
30123        1        0       157    268         2         0     0      3  0
6679         1        2       199     -1         0         3     0      8  0
25514        1        0        76     -1         0         3     0      9  0
39215        1        0       695     -1         0         3     0      8  0


# Yes data only

In [9]:
data_train=df3.copy(deep=True)
data_train=data_train[data_train.pred==1]
label_data=data_train['month']
train_y=data_train['y']
train_pred=data_train['pred']
data_train=data_train.drop('month', axis=1)
data_train=data_train.drop('y', axis=1)
data_train=data_train.drop('pred', axis=1)

data_train.head()

from sklearn.svm import SVC
gnb = SVC()
gnb.fit(data_train, label_data)

predictions = gnb.predict(data_train)
data_train['month_pred']=predictions

data_train['y']=train_y
data_train['pred']=train_pred
data_train['month']=train_month

data_train.head()


,housing,contact,duration,pdays,previous,poutcome,month_pred,y,pred,month
37065,1,0,628,-1,0,3,8,0,1,8
18594,1,0,744,-1,0,3,5,0,1,5
31457,0,0,829,-1,0,3,5,0,1,7
20644,1,0,454,-1,0,3,1,0,1,1
28934,0,0,608,-1,0,3,8,0,1,3


In [10]:
data_train_n=df3.copy(deep=True)
data_train_n=data_train_n[data_train_n.pred==0]
label_data_n=data_train_n['month']
train_y_n=data_train_n['y']
train_pred_n=data_train_n['pred']
data_train_n=data_train_n.drop('month', axis=1)
data_train_n=data_train_n.drop('y', axis=1)
data_train_n=data_train_n.drop('pred', axis=1)
data_train_n.head()


gnb_n = SVC()
gnb_n.fit(data_train_n, label_data_n)

predictions_n = gnb_n.predict(data_train_n)

data_train_n.head()

data_train_n['month_pred']=predictions_n

data_train_n['y']=train_y_n
data_train_n['pred']=train_pred_n
data_train_n['month']=label_data_n
data_train_n.head()




,housing,contact,duration,pdays,previous,poutcome,month_pred,y,pred,month
11139,1,2,34,-1,0,3,8,0,0,6
30123,1,0,157,268,2,0,3,0,0,3
6679,1,2,199,-1,0,3,8,0,0,8
25514,1,0,76,-1,0,3,5,0,0,9
39215,1,0,695,-1,0,3,5,0,0,8


# Testing

In [11]:
data_test_y=data_test_Backup.copy(deep=True)
#data_test_y=data_test_y[data_train_n.pred==0]
data_label_y=data_test_Backup['y']
data_label_month=data_test_Backup['month']
data_test_y=data_test_y.drop('month', axis=1)
data_test_y=data_test_y.drop('y', axis=1)
#data_test_y.head()
#data_test_kmeans['Clust']= kmeans.predict(data_train_kmeans)


a=rfc.predict(data_test_y)

data_test_y['pre']=a
data_test_y['y']=data_label_y
data_test_y['month']=data_label_month
data_test_y.head()

,housing,contact,duration,pdays,previous,poutcome,pre,y,month
14001,1,0,91,-1,0,3,0,0,5
32046,1,0,1073,-1,0,3,1,1,0
13318,0,1,243,-1,0,3,0,0,5
42991,1,0,105,188,2,0,0,0,3
14237,1,0,332,-1,0,3,0,0,5


In [12]:
df7=data_test_y.copy(deep=True)
df8 = df7[df7['pre'] == 1]
df8.head()

label_df8=df8['month']

df8_pred=df8['pre']
df8.head()
df8=df8.drop('month', axis=1)
df8=df8.drop('pre', axis=1)
df8=df8.drop('y', axis=1)
df8.head()

predictions = gnb.predict(df8)

    
df8['pred_month']=predictions
df8['pre']=df8_pred
df8['month']=label_df8

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

PrecisionScore=precision_score(df8['month'], df8['pred_month'], average='weighted')
RecallScore=recall_score(df8['month'], df8['pred_month'], average='weighted')
FScore=f1_score(df8['month'], df8['pred_month'], average='weighted')
    

print("precision_score: ",PrecisionScore)
print("Recall_score: ",RecallScore)
print("F1 score: ",FScore)


precision_score:  0.19763907305185405
Recall_score:  0.25780189959294436
F1 score:  0.19510721326670505


/Users/harneetkaur/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harneetkaur/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# No data only

In [13]:
df7=data_test_y.copy(deep=True)
df8 = df7[df7['pre'] == 0]
df8.head()

label_df8=df8['month']

df8_pred=df8['pre']
df8.head()
df8=df8.drop('month', axis=1)
df8=df8.drop('pre', axis=1)
df8=df8.drop('y', axis=1)
df8.head()

predictions = gnb.predict(df8)

    
df8['pred_month']=predictions
df8['pre']=df8_pred
df8['month']=label_df8

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

PrecisionScore=precision_score(df8['month'], df8['pred_month'], average='weighted')
RecallScore=recall_score(df8['month'], df8['pred_month'], average='weighted')
FScore=f1_score(df8['month'], df8['pred_month'], average='weighted')
    

print("precision_score: ",PrecisionScore)
print("Recall_score: ",RecallScore)
print("F1 score: ",FScore)
print()


precision_score:  0.1805605243689081
Recall_score:  0.2755839152419937
F1 score:  0.1746952414601827



/Users/harneetkaur/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/harneetkaur/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
